# Ollama + OpenAI + Python

## 1. Määritä mallin nimi

Jos olet ladannut eri mallin kuin "phi3:mini", vaihda arvo alla olevaan soluun. Tätä muuttujaa käytetään koodissa koko muistikirjan ajan.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Aseta OpenAI-asiakas

Tyypillisesti OpenAI-asiakasta käytetään OpenAI.comin tai Azure OpenAI:n kanssa suurten kielimallien kanssa vuorovaikuttamiseen.  
Kuitenkin sitä voidaan käyttää myös Ollaman kanssa, koska Ollama tarjoaa OpenAI-yhteensopivan päätepisteen osoitteessa "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Luo keskusteluvastaus

Nyt voimme käyttää OpenAI SDK:ta luodaksemme vastauksen keskusteluun. Tämä pyyntö pitäisi luoda haikun kissoista:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Kehotusmuotoilu

Ensimmäistä viestiä, joka lähetetään kielimallille, kutsutaan "järjestelmäviestiksi" tai "järjestelmäkehotukseksi", ja se määrittää mallin yleiset ohjeet.  
Voit antaa oman mukautetun järjestelmäkehotuksen ohjataksesi kielimallia tuottamaan erilaisia vastauksia.  
Muokkaa alla olevaa `SYSTEM_MESSAGE`-kohtaa niin, että malli vastaa kuin suosikkielokuvasi tai -TV-sarjasi hahmo, tai hae inspiraatiota muihin järjestelmäkehotuksiin [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Kun olet mukauttanut järjestelmäviestin, anna ensimmäinen käyttäjän kysymys kohtaan `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Muutamia esimerkkitapauksia

Toinen tapa ohjata kielimallia on antaa "muutamia esimerkkejä", eli sarja kysymys/vastauspareja, jotka osoittavat, miten sen tulisi vastata.

Alla oleva esimerkki pyrkii saamaan kielimallin toimimaan opetusavustajana antamalla muutamia esimerkkejä kysymyksistä ja vastauksista, joita opetusavustaja voisi antaa, ja sitten esittää mallille kysymyksen, jonka opiskelija voisi kysyä.

Kokeile ensin, ja muokkaa sitten `SYSTEM_MESSAGE`, `EXAMPLES` ja `USER_MESSAGE` uutta tilannetta varten.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Tiedonhakuun perustuva generointi

RAG (Retrieval Augmented Generation) on tekniikka, jonka avulla kielimalli saadaan vastaamaan kysymyksiin tarkasti tietyn alan osalta. Tämä tapahtuu ensin hakemalla asiaankuuluvaa tietoa tietolähteestä ja sitten tuottamalla vastaus tämän tiedon perusteella.

Olemme toimittaneet paikallisen CSV-tiedoston, joka sisältää tietoa hybridiautoista. Alla oleva koodi lukee CSV-tiedoston, etsii käyttäjän kysymykseen sopivia osumia ja tuottaa vastauksen löydetyn tiedon perusteella. Huomaa, että tämä vie enemmän aikaa kuin aiemmat esimerkit, koska mallille lähetetään enemmän dataa. Jos huomaat, että vastaus ei vieläkään perustu kunnolla dataan, voit kokeilla järjestelmän hienosäätöä tai muita malleja. Yleisesti ottaen RAG toimii paremmin joko suurempien mallien tai hienosäädettyjen SLM-versioiden kanssa.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
